<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Подготока-обучающей-и-валидационной-выборки" data-toc-modified-id="Подготока-обучающей-и-валидационной-выборки-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>Подготока обучающей и валидационной выборки</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.0.2"><span class="toc-item-num">2.0.2&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.0.3"><span class="toc-item-num">2.0.3&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.0.4"><span class="toc-item-num">2.0.4&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Градиентный-бустинг" data-toc-modified-id="Градиентный-бустинг-2.0.5"><span class="toc-item-num">2.0.5&nbsp;&nbsp;</span>Градиентный бустинг</a></span></li><li><span><a href="#PassiveAgressiveClassifier" data-toc-modified-id="PassiveAgressiveClassifier-2.0.6"><span class="toc-item-num">2.0.6&nbsp;&nbsp;</span>PassiveAgressiveClassifier</a></span></li></ul></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import re
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import math
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import PassiveAggressiveClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.dummy import DummyRegressor
from sklearn.metrics import accuracy_score 
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
from scipy import stats as st
from numpy.random import RandomState
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer 
from nltk.corpus import stopwords 
import nltk
from sklearn.linear_model import LogisticRegression 
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords') 
from sklearn.feature_extraction.text import TfidfVectorizer 
from pymystem3 import Mystem
m = Mystem() 
from nltk.stem import WordNetLemmatizer 
from lightgbm import LGBMClassifier
from sklearn.dummy import DummyClassifier

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv('toxic_comments.csv')
corpus = data['text'].values.astype('U')
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
lemmatizer =  nltk.stem.WordNetLemmatizer()
tokenizer=nltk.tokenize.WhitespaceTokenizer()


Напишем функцию для токкенизации, лемматизации и очищения текстов:

In [4]:
def clear_text(text):
    return " ".join(re.sub(r'[^a-zA-Z ]', ' ', text).split())  

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(w,'v') for w in tokenizer.tokenize(text)])

In [5]:
#Проверим работу функции
print(lemmatize_text(clear_text("years, cars")))
print(corpus[14])
lemmatize_text(clear_text(corpus[14]))

years cars
Oh, and the girl above started her arguments with me. She stuck her nose where it doesn't belong. I believe the argument was between me and Yvesnimmo. But like I said, the situation was settled and I apologized. Thanks,


'Oh and the girl above start her arguments with me She stick her nose where it doesn t belong I believe the argument be between me and Yvesnimmo But like I say the situation be settle and I apologize Thanks'

In [6]:
%%time
data['lemm_text']=1
for i in range(data.shape[0]):
    data['lemm_text'][i]= lemmatize_text(clear_text(corpus[i]))

Wall time: 3min 57s


In [7]:
data.sample(10)

,text,toxic,lemm_text
97762,86.134.32.13\nThis user is vandalising Ajith a...,0,This user be vandalise Ajith and Vijay actor a...
37599,I ask you... why would anyone need to know inf...,0,I ask you why would anyone need to know inform...
34872,Why \nHowcome u want the Cibola Caounty Beacon...,0,Why Howcome u want the Cibola Caounty Beacon a...
4235,"""\nno seriousy. the only thing i think actual...",1,no seriousy the only thing i think actually do...
14356,It's my perception that the number of editors ...,0,It s my perception that the number of editors ...
50672,Big thanks to you and Red for helping promote ...,0,Big thank to you and Red for help promote this
123591,RepublicanJacobite thinks that he knows everyt...,0,RepublicanJacobite think that he know everythi...
22165,"""\n\n Umm...unblock maybe? \n\nI was just wond...",0,Umm unblock maybe I be just wonder if you coul...
45060,"""\nThe practice: 1. only (central) governmenta...",0,The practice only central governmental body or...
106256,Age\n\nCurrently this article inaccurately sta...,0,Age Currently this article inaccurately state ...


## Обучение

#### Подготока обучающей и валидационной выборки

In [8]:
features = data['lemm_text']
target = data['toxic']
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state = 12345, test_size=0.2)

In [9]:
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
tf_idf = count_tf_idf.fit_transform(features_train) 
tf_idf2 = count_tf_idf.transform(features_test)

#### Логистическая регрессия

In [10]:
%%time
parameters = {'C':range(1,10,1)}

log_regr_rand=RandomizedSearchCV(LogisticRegression(class_weight='balanced',penalty='l2'), parameters, cv=5, random_state=12345,scoring='f1')
log_regr_rand.fit(tf_idf, target_train)
print('Лучшие параметры: ',log_regr_rand.best_params_)
log_regression_model=log_regr_rand.best_estimator_


Лучшие параметры:  {'C': 5}
Wall time: 2min 51s


In [11]:
%%time
prediction = log_regression_model.predict(tf_idf2)
log_regression_result=f1_score(target_test, prediction)
print('F1 score for logistiv regression',log_regression_result)

F1 score for logistiv regression 0.7611290549080185
Wall time: 15 ms


#### Дерево решений

In [12]:
%%time

parameters = {'max_depth':range(1,100,25)}
dcs_tree_rand_search=RandomizedSearchCV(DecisionTreeClassifier(class_weight='balanced'), parameters, cv=5, random_state=12345,scoring='f1')
dcs_tree_rand_search.fit(tf_idf, target_train)
print('Лучшие параметры: ',dcs_tree_rand_search.best_params_)
model_des_tr_class=dcs_tree_rand_search.best_estimator_



Лучшие параметры:  {'max_depth': 51}
Wall time: 10min 57s


In [13]:
dcs_tree_rand_search.best_score_

0.6401575687762022

In [14]:
prediction = model_des_tr_class.predict(tf_idf2)
model_des_tr_class_result=f1_score(target_test, prediction)
print('F1 для дерева решений',model_des_tr_class_result)

F1 для дерева решений 0.6172133614204033


#### Случайный лес

In [15]:
#%%time
#parameters = {'max_depth':range(1,300,50)}
#rnd_for_clas_search=RandomizedSearchCV(RandomForestClassifier(class_weight='balanced'), parameters, cv=5, random_state=12345,scoring='f1', n_jobs=-1)
#rnd_for_clas_search.fit(tf_idf, target_train)
#print('Лучшие параметры: ',rnd_for_clas_search.best_params_)
#model_des_tr_class=rnd_for_clas_search.best_estimator_


In [16]:
%%time
model_des_tr_class=RandomForestClassifier(class_weight='balanced',max_depth=251)
model_des_tr_class.fit(tf_idf, target_train)

Wall time: 5min 7s


RandomForestClassifier(class_weight='balanced', max_depth=251)

In [17]:
predictions = model_des_tr_class.predict(tf_idf2)
model_des_rnd_fr_class_result=f1_score(target_test, predictions)
print('F1 для случайного леса',model_des_rnd_fr_class_result)

F1 для случайного леса 0.593381389252949


#### Градиентный бустинг

In [18]:
#Градиентный бустинг на сетевом юпитере у меня зависает,ядро падает, взял параметры с локального юпитера + скриншот ниже.
#Лучшие параметры:  {'num_leaves': 30, 'n_estimators': 361}
#Wall time: 9min 57s

In [19]:
%%time
LGBM_class=LGBMClassifier(num_leaves=30,n_estimators=361)
LGBM_class.fit(tf_idf, target_train)
predictions = LGBM_class.predict(tf_idf2)
print(f1_score(target_test, predictions))

0.7760884588804423
Wall time: 39.7 s


In [20]:
%%time
parameters = {'n_estimators':range(1,400,10), 'num_leaves':range(10,50,5)}
lgbm_rand_frst=RandomizedSearchCV(LGBMClassifier(random_state=12345), parameters, cv=5, random_state=12345,scoring='f1', n_jobs=-1)
lgbm_rand_frst.fit(tf_idf, target_train)
print('Лучшие параметры: ',lgbm_rand_frst.best_params_)
model_lgb=lgbm_rand_frst.best_estimator_

Лучшие параметры:  {'num_leaves': 40, 'n_estimators': 291}
Wall time: 9min 36s


In [21]:
prediction=model_lgb.predict(tf_idf2)
model_lgb_result=f1_score(target_test, predictions)

In [25]:
LGBM_result=model_lgb_result
LGBM_result

0.7760884588804423

#### PassiveAgressiveClassifier

In [22]:
%%time
model = PassiveAggressiveClassifier(random_state=12345, max_iter=5)
model.fit(tf_idf, target_train)
prediction = model.predict(tf_idf2)
pac_result=f1_score(target_test, prediction)

Wall time: 233 ms


In [23]:
print('F1-score for PAC:',pac_result)

F1-score for PAC: 0.7700640078778928


## Выводы

In [26]:
columns = ['F1']
values_table=[log_regression_result,model_des_tr_class_result,model_des_rnd_fr_class_result,LGBM_result,pac_result]
values_table_df=pd.DataFrame(data=values_table,columns=columns, index =['LogisticRegression', 'DecisionTreeClassifier','RandomForestClassifier','LGBMClassifier','PassiveAggressiveClassifier'] )
values_table_df

,F1
LogisticRegression,0.761129
DecisionTreeClassifier,0.617213
RandomForestClassifier,0.593381
LGBMClassifier,0.776088
PassiveAggressiveClassifier,0.770064


1.Лучшие результаты показала модель PassiveAgressiveClassifier, логистическая регрессия и градиентный бустинг.<br>
2.Деревья решений с жаднм алгоритмом покзали результат худший, чем линейные модели.